# Exam question - course outcome matching

Author: Mertcan Atay<br>
Repo: https://github.com/mertcanfsm/exam_ai<br>

Using limited text data to predict class label.<br>

<ul>
    <li>Dataset size:</li>
        <ul>
            <li>Very small (&lt 20 samples / Turkish)</li>
            <li>Small (&lt 150 samples / English)</li>
        </ul>
    <li>Word embeddings</li>
        <ul>
            <li>Tfidf</li>
            <li>Word2vec</li>
        </ul>
    <li>Model selection</li>
        <ul>
            <li>Train-test split</li>
            <li>Leave one out</li>
        </ul>
    <li>Machine learning models</li>
        <ul>
            <li>MultinomialNB</li>
            <li>LogisticRegression</li>
            <li>LinearSVC</li>
            <li>RandomForestClassifier</li>
        </ul>
</ul>

In [2]:
import nltk
import pickle
import numpy as np
import pandas as pd
from preprocess import preprocess
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

Choose one of the following word2vec models (if word2vec is choosen):

In [3]:
# Small Turkish word2vec model (https://www.kaggle.com/murats/word2vec-application-on-turkish-newspaper/notebook)
w2v = Word2Vec.load("word2vec_small").wv
keys = w2v.index_to_key
len(keys)

69133

In [4]:
# Big Turkish word2vec model (https://github.com/akoksal/Turkish-Word2Vec)
w2v = Word2Vec().wv.load_word2vec_format("word2vec_big", binary=True)
keys = w2v.index_to_key
len(keys)

412457

In [5]:
# English word2vec model (https://github.com/eyaler/word2vec-slim)
w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300-SLIM.bin", binary=True)
keys = w2v.index_to_key
len(keys)

299567

Word2vec preprocessing for sentences:

In [ ]:
def prepare_w2v(tokens,w2v,keys):
    # Remove words not in word2vec model
    for i in range(len(tokens)):
        word = tokens[i]
        if word in keys:
            tokens[i] = w2v[word]
        else:
            tokens[i] = None
    tokens = list(filter(lambda x: hasattr(x,"__len__"), tokens))

    # Calculate the average vector of the sentence words
    vec_avg = []
    if len(tokens) == 0:
        return np.zeros(w2v.vector_size)
    dim = len(tokens[0])
    sen_len = len(tokens)
    for i in range(dim):
        total = 0
        for j in range(sen_len):
            total += tokens[j][i]
        vec_avg.append(total / sen_len)
    return vec_avg

Choose one of the two datasets:

<b>1- Example exam questions and their respective course outcome IDs.</b>

In [ ]:
X = np.array([
    'Verilen 5x5’lük dizi içinde her satırda -1 özel rakamı arasında kalan sayıların toplamını ekrana yazdıran algoritmanın kodunu yazınız ve akış diyagramını çiziniz. Satırda -1 yoksa 0 yazacaktır. (ipucu ilk satırda 9 ikinci satırda 18 yazacak.) a şıkkındaki algoritmayı sadece -1 geçen (toplamı 0 olmayan) satırları bir Arraylist’e aktaracak şekilde düzenleyiniz.',
    'Aşağıdaki sınıf yapısını kodlayınız. İsmi verilen metotlar sorulardaki ihtiyaçlara göre düzenlenmelidir. Metot içinde nesnenin gerekli özellikleri kullanılmalıdır.  Personel  adi alanını ilk atamadan sonra sabit yapınız. Her sınıf kendi ismini taşıyan paketteyse BirimUnvan sınıfı içinde Memur sınıfına erişmek için gerekli import kodunu yazınız. toplamMaas özelliğini sınıf değişkeni yapınız. Personel sınıfı unvanlar özelliğini unvan nesnesi tutan bir array olarak gerçekleştiriniz. Dizide 10 adet unvan saklanabilecek şekilde ilk değer veriniz.  Personel sınıfı adresler özelliğini String tutan bir ArrayList olarak gerçekleştiriniz. getUnvanMaas metodunu YoneticiUnvan sınıflarda maaş değerinin %20 fazlasını döndürecek şekilde özelleştiriniz(override). YoneticiUnvan ve BirimUnvan türlerinde iki parametre alan, bunları bir diziye ekleyen ve bu diziyi döndüren bir metodun tanımını(imzasını) yazınız. Metodun gerçekleştirilmesine gerek yoktur.',
    'İkinci Sorudaki Modeli kullanarak getPersonelMaas metodunu kullanarak unvanlar dizisi üzerinden personele ödenen toplam maaşı yazdırınız. IMaasHesap isimli bir interface oluşturunuz.  Bu interface’de unvanEkle(Unvan u) metodu olacaktır. Personel sınıfı içerisinde IMaasHesap interface’ini gerçekleştiren bir statik gömülü sınıf oluşturunuz. Bu gömülü sınıfta personelin unvanlar dizisine her unvan eklendiğinde toplam maaşı unvanın maas özelliği kadar arttıracak kodu yazınız. Bu metotları test sınıfından çağırınız.  Personel sınıfına parametre olarak gönderilen adresi adresler ArrayList’ine ekleyecek şekilde bir yapıcı (costructor) metodu ekleyiniz. Memur sınıfı içerisinde bir iç sınıf kullanarak adresler dizisindeki adresleri satır satır yazdıran bir metodu kodlayınız. Bu metotları test sınıfından çağıran kodu yazınız.',
    'Aşağıdaki kodun doğru çalışması için IX arayüzü ve sinifA’yı metotlarının sadece tanımları(imzaları) olarak şekilde yazınız. SinifA IX arayüzünü gerçekleştirmektedir.',
    'Gömülü sınıf, iç sınıf ve yerel sınıf kavramlarını tanımlayınız ve kısaca örnekleyiniz. Soyut sınıfın ara yüze(interface) göre üstünlüklerini açıklayınız Çok biçimlilik ve metot aşırı yükleme (overloading) ilişkisini açıklayınız.',
    'Verilen dizi içinde yan yana aynı değerlere sahip sütunların değerlerini yazdıran algoritmanın kodunu yazınız ve akış diyagramını çiziniz. Algoritma satır sayısı çok fazla olan durumlarda da çalışmalıdır. Sınırlara dikkat ediniz. İlk satırla aynı değerleri taşıyan sütunların adedini ilk sütunla yan yana olmasa da bulan algoritmayı yazınız.',
    'Aşağıdaki sınıf yapısını kodlayınız. İsmi verilen metotlar sorulardaki ihtiyaçlara göre düzenlenmelidir. Metot içinde nesnenin gerekli özellikleri kullanılmalıdır. Tüm şıklar kodun yanında belirtilmelidir. Mesai sınıfını alt sınıf üretilemez hale getiriniz. Kamu sınıfına kamuNo alanına değer atayan bir yapıcı metot ekleyiniz. Hastane sınıfına arayüz yapmak için modelde ne tür değişiklikler yapılmalıdır. Hizmetlinin görev niteliğini sadece kendi sınıfı içinde gözükebilen hale getiriniz. Hastane adres niteliğini hiçbir örnekte ve sınıf düzeyinde değiştirilemez yapınız. Kisi sınıfını soyut sınıf yapmak için modelde ne tür değişiklikler yapılmalıdır.',
    'İkinci Sorudaki Modeli kullanarak Doktor sınıfına herhangi bir sayıda Mesai nesnesi alan ve bunları mesaiListesine ekleyen bir metot ekleyiniz. getKısıUcret() metodu ile kişinin çalıştığı aylar üzerinden kişiye ödenen toplam ücreti bulunuz. mesiaGoster() metodunu kullanarak Doktor sadece özel hastanede çalışıyorsa toplam mesaisini döndürecek şekilde gerçekleştiriniz. mesaiUcreti saat cinsindendir. (İpucu: Nesne türü bulmak kullanılmalıdır.) getKısıUcret() metodunu Doktor sınıfında override ederek toplam mesai de eklenmiş ücreti bulacak hale getiriniz.  Hastane sınıfına kaç adet doktorun kamu ve özel hastanelerde çalıştığını bulan bir metot ekleyiniz.',
    'Aşağıdaki kodun doğru çalışması için gerekli arayüz ve sınıfları tanımlayınız? IKisi bir arayüzdür.  Kısaca açıklayınız. Metotların aldıkları değişken türlerine dikkat ediniz.',
    'Kapsüllemeyi (encapsulation) tanımlayınız. Faydalarını açıklayınız. Aşağı ve yukarı dönüşüm işlemlerini kısaca tanımlayınız. Niçin kullanıldıklarını açıklayınız. Dinamik metot bağlama ve çok biçimlilik ilişkisini açıklayınız.',
    'Verilen 5x5’lik dizi içinde yan yana numaraların adet ve toplamını gösteren algoritmanın kodunu yazınız ve akış diyagramını çiziniz. Sınırlara dikkat ediniz. Yan yana rakamların toplamlarını bir diziye aktarınız. Örneğin dizide 1 rakamı için 3, 85 rakamı için 10 tutulmalıdır. İpucu rakamlar dizi indisi olarak kullanılabilir.',
    'Aşağıdaki sınıf yapısını kodlayınız. İsmi verilen metotlar sorulardaki ihtiyaçlara göre düzenlenmelidir. Metot içinde nesnenin gerekli özellikleri kullanılmalıdır. Tüm şıklar kodun yanında belirtilmelidir. IDers arayüzünü tüm paketlerde görünür yapınız. Oğrenci sınıfı ayrı bir kisi.ogrenci paketindeyse bunu Matematik sınıfından referans etmek için gerekli import kodunuz yazınız.  Bolum  özelliğinin tüm malzeme örneklerinde (instance) aynı değeri taşıması için gerekli kodu yazınız.  Matematik sınıfındaki adi özelliğini tüm paketlerdeki alt sınıflarda görünür yapmak için gerekli kodu yazınız. Alt sınıflarda üst sınıf özelliğine ulaşan örnek bir kod yazınız. Ogrenci sınıfında adi niteliğini alan bir yapıcı metot ekleyiniz. Ogrenci sınıfindan bilgisayarOğrencisi sınıfı kalıtılırsa yapıcıların doğru çalışması için yapılması gerekenleri yazınız.',
    'İkinci Sorudaki Modeli kullanarak OgrenciDers sınıfındaki  getNotOrtalama()  metodunu dersin ortalama notunu bulacak şekilde gerçekleştiriniz.  getHarf Not metodu bu sınıfta gerçekleştirilmemelidir. Ogrenci sınıfındaki  getNotOrtalama()  metodunu öğrencinin tüm derslerdeki ortalama notunu bulacak şekilde gerçekleştiriniz.   Matematik sınıfında getNotOrtalama metodunu öğrencinin tüm derslerden aldığı not ortalamasını odev notunu da ekleyecek şekilde özelleştiriniz.  IDers arayüzündeki getNotOrtalama() metodunu not sistemini de alacak şekilde overload ediniz.  VeriYapilari sınıfında getHaftNot metodunu  NotSistemi nesnesi alacak, NotSistemi yüzlükse ve ortalaması  90 üzeri öğrencileri için  AA,  diğer durumda XX döndürecek şekilde gerçekleştiriniz. Ogrenciye ders ve not ekleyerek b şıkkını test eden kodu yazınız.',
    'Aşağıdaki kodun doğru çalışması için gerekli sınıfları tanımlayınız? Kısaca açıklayınız.',
    'Sınıflar arası kalıtım ve arayüz kullanımını karşılaştırınız. Bunlar birbiri yerine kullanılabilir mi ve kullanılırsa ne tür olumsuz olumsuz durumlar oluşur belirtiniz.  Gömülü sınıflarda kalıtım kullanımının sağlayacağı faydaları açıklayınız. Arayüzü sınıftan ayıran özellikleri açıklayınız.'
])
y = np.array([4,3,1,2,0,4,3,1,2,1,4,3,1,2,0])
language = 'turkish'
course_name = 'BLM19103'
stem = False

<b>2- For performance testing purposes, a bigger text dataset for classification purposes.</b>

In [ ]:
df = pd.DataFrame(pd.read_csv('questions_processed.csv'))
X = df[df.columns[0]].to_numpy()
y = df[df.columns[1]].to_numpy()
language = 'english'
course_name = 'GCSE_CS'
stem = True
df

Example prediction sentence:

In [ ]:
sentence = "Verilen 5x5’lük dizi içinde her satırda -1 özel rakamı arasında kalan sayıların toplamını ekrana yazdıran algoritmanın kodunu yazınız ve akış diyagramını çiziniz. Satırda -1 yoksa 0 yazacaktır. (ipucu ilk satırda 9 ikinci satırda 18 yazacak.) Algoritma satır sayısı çok fazla olan durumlarda da çalışmalıdır. Sınırlara dikkat ediniz."

Choose tfidf or word2vec and preprocess text data:

In [ ]:
tfidf = True

Data preprocessing:

In [ ]:
if tfidf:
    X = [' '.join(preprocess(text,language,stem=stem)) for text in X]
else:
    X = [prepare_w2v(preprocess(text,language,stem=stem),w2v,keys) for text in X]

X = np.array(X)

# Remove empty vectors
if not tfidf:
    is_zeros = []
    for x in X:
        is_zeros.append(np.logical_not(np.all(x == 0)))
    X = X[is_zeros]
    y = y[is_zeros]

The train codes for both tfidf and word2vec training:

In [ ]:
def train_tfidf(model,X_train,X_test,y_train,y_test):
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    X_test_counts = count_vect.transform(X_test)

    tfidf_transformer = TfidfTransformer()
    X_trained = tfidf_transformer.fit_transform(X_train_counts)

    clf = model.fit(X_trained, y_train)

    y_pred = clf.predict(X_test_counts)
    accuracy = (y_test == y_pred).sum() / X_test_counts.shape[0]
    
    return accuracy

def train_word2vec(model,X_train,X_test,y_train,y_test):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    clf = model.fit(X_train_scaled, y_train)

    X_test_scaled = scaler.transform(X_test)
    y_pred = clf.predict(X_test_scaled)
    accuracy = (y_test == y_pred).sum() / X_test_scaled.shape[0]
    
    return accuracy

The classification models to compare:

In [ ]:
models = [
    RandomForestClassifier(random_state=0),
    LinearSVC(random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

Finally, run the code by choosing one of the two data splitting methods:

<b>1- Train-test split</b>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

for model in models:
    accuracy = 0
    model_name = model.__class__.__name__
    if tfidf:
        accuracy = train_tfidf(model, X_train, X_test, y_train, y_test)
    else:
        accuracy = train_word2vec(model, X_train, X_test, y_train, y_test)
    print(model_name + " accuracy: " + str(accuracy))

<b>2- Leave one out</b>

In [ ]:
for model in models:
    i = 0
    accuracy = 0
    model_name = model.__class__.__name__
    for train_index, test_index in LeaveOneOut().split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        if tfidf:
            acc = train_tfidf(model, X_train, X_test, y_train, y_test)
        else:
            acc = train_word2vec(model, X_train, X_test, y_train, y_test)
        accuracy += acc
        i += 1
    print(model_name + " accuracy: " + str(accuracy/i))

<b>Bonus: Missing input words in word2vec model calculator</b>

In [ ]:
def word2vec_words_found(X,language,stem):
    found = 0
    missing = 0
    for sentence in X:
        for word in preprocess(sentence,language,stem=stem):
            if word in keys:
                found += 1
            else:
                missing += 1
    return found/(found+missing)

print("Found (with stemming): " + str(word2vec_words_found(X,language,True)))
print("Found (without stemming): " + str(word2vec_words_found(X,language,False)))

# Results

<i>(Below results were achieved with MultinomialNB model.)</i>
<h2>Tfidf</h2>
<h4>Turkish</h4>
+ With stemming<br>
- Train-test split accuracy: 0.25<br>
- Leave-one-out split accuracy: 0.6166666666666667<br>
+ Without stemming<br>
- Train-test split accuracy: 0.5<br>
- Leave-one-out split accuracy: 0.7666666666666667
<h4>English</h4>
+ With stemming<br>
- Train-test split accuracy: 0.6060606060606061<br>
- Leave-one-out split accuracy: 0.6305806153134398<br>
+ Without stemming<br>
- Train-test split accuracy: 0.6060606060606061<br>
- Leave-one-out split accuracy: 0.6382142030996993<br>

<h2>Word2vec</h2>
<h4>Small Turkish</h4>
+ With stemming<br>
- Train-test split accuracy: 0.25<br>
- Leave-one-out split accuracy: 0.48333333333333334<br>
+ Without stemming<br>
- Train-test split accuracy: 0.25<br>
- Leave-one-out split accuracy: 0.75
<h4>Big Turkish</h4>
+ With stemming<br>
- Train-test split accuracy: 0.5<br>
- Leave-one-out split accuracy: 0.7666666666666667<br>
+ Without stemming<br>
- Train-test split accuracy: 0.25<br>
- Leave-one-out split accuracy: 0.6166666666666667
<h4>English</h4>
+ With stemming<br>
- Train-test split accuracy: 0.75<br>
- Leave-one-out split accuracy: 0.6377952755905512<br>
+ Without stemming<br>
- Train-test split accuracy: 0.7272727272727273<br>
- Leave-one-out split accuracy: 0.676923076923077<br>

<h2>Available Word2vec words</h2>
Small Turkish<br>
- With stemming: 0.6872536136662286<br>
- Without stemming: 0.668856767411301<br>
Big Turkish<br>
- With stemming: 0.8055190538764783<br>
- Without stemming: 0.8620236530880421<br>
English<br>
- With stemming: 0.7068965517241379<br>
- Without stemming: 0.9379310344827586<br>

<h2>Word2vec model size/speed</h2>
(refer to word2vec_speed.ipynb)<br>
- Small Turkish: 86MB / 1.7109315395355225 seconds<br>
- Big Turkish: 633MB / 3.560025215148926 seconds<br>
- English model: 362MB / 2.4627420902252197 seconds<br>

# Conclusions
The best result we got with the English dataset was using stemming, word2vec and train-test data split (75% accuracy).<br>
In the Turkish dataset we got two ways of achieving best results (76% accuracy):<br>
1- No stemming, tfidf, leave-one-out<br>
2- Stemming, big word2vec model, leave-one-out<br>
As the first method is both faster (no stemming and word2vec lookup required) and takes less space (no big word2vec model required), I decided that the first method is the best one for the Turkish dataset.<br>

The above results were all achieved with MultinomialNB. Now I will use different classification models and see which one will give the best results (using the best preprocessing steps I listed above for both datasets.)<br>

<b>Turkish, no stemming, tfidf, leave-one-out</b><br>
RandomForestClassifier accuracy: 0.7333333333333333<br>
LinearSVC accuracy: 0.7333333333333333<br>
MultinomialNB accuracy: 0.7333333333333333<br>
LogisticRegression accuracy: 0.7333333333333333<br>

<b>English, stemming, word2vec, train-test split</b><br>
RandomForestClassifier accuracy: 0.6875<br>
LinearSVC accuracy: 0.65625<br>
MultinomialNB accuracy: 0.75<br>
LogisticRegression accuracy: 0.6875<br>

As we can see from above results, the classification model doesn't matter for the Turkish dataset as all training algorithms achieve the same results.<br>

As for the English dataset, we can see that MultinomialNB gave us the best results compared to other models.<br>